# PROJE: Rating Product & Sorting Reviews in Amazon

###################################################

### İş Problemi


E-ticaretteki en önemli problemlerden bir tanesi ürünlere satış sonrası verilen puanların doğru şekilde hesaplanmasıdır.

Bu problemin çözümü e-ticaret sitesi için daha fazla müşteri memnuniyeti sağlamak, satıcılar için ürünün öne çıkması ve satın alanlar için sorunsuz bir alışveriş deneyimi demektir. Bir diğer problem ise ürünlere verilen yorumların doğru bir şekilde sıralanması olarak karşımıza çıkmaktadır. 

Yanıltıcı yorumların öne çıkması ürünün satışını doğrudan etkileyeceğinden dolayı hem maddi kayıp hem de müşteri kaybına neden olacaktır. Bu 2 temel problemin çözümünde e-ticaret sitesi ve satıcılar satışlarını arttırırken müşteriler ise satın alma yolculuğunu sorunsuz olarak tamamlayacaktır.

###################################################

In [1]:
###################################################
# Veri Seti Hikayesi

# Amazon ürün verilerini içeren bu veri seti ürün kategorileri ile çeşitli metadataları içermektedir.
# Elektronik kategorisindeki en fazla yorum alan ürünün kullanıcı puanları ve yorumları vardır.

# Değişkenler;

# reviewerID: Kullanıcı ID’si
# asin: Ürün ID’si
# reviewerName: Kullanıcı Adı
# helpful: Faydalı değerlendirme derecesi
# reviewText: Değerlendirme
# overall: Ürün rating’i
# summary: Değerlendirme özeti
# unixReviewTime: Değerlendirme zamanı
# reviewTime: Değerlendirme zamanı Raw
# day_diff: Değerlendirmeden itibaren geçen gün sayısı
# helpful_yes: Değerlendirmenin faydalı bulunma sayısı
# total_vote: Değerlendirmeye verilen oy sayısı

In [2]:
# To Do List;

# GÖREV 1: Average Rating'i Güncel Yorumlara Göre Hesaplayınız ve Var Olan Average Rating ile kıyaslayalım.

# Görev 2: Ürün için Ürün Detay Sayfasında Görüntülenecek 20 Review'i belirleyelim.

In [4]:
# Kütüphanelerimizi import edelim;

import matplotlib.pyplot as plt
import pandas as pd
import math
import scipy.stats as st

# Pandas ayarlamalarımızı yapalım;

pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', 10)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('display.float_format', lambda x: '%.2f' % x)

### Görev 1:

    Veri setinde kullanıcılar bir ürüne puanlar vermiş ve yorumlar yapmıştır.
    Bu görevde amacımız verilen puanları tarihe göre ağırlıklandırarak değerlendirmek.
    İlk ortalama puan ile elde edilecek tarihe göre ağırlıklı puanın karşılaştırılması gerekmektedir.

In [13]:
# Veri setimizi okutalım;

df = pd.read_csv("datasets/amazon_review.csv")

# Verimize hızlı bir bakış atalım;
df.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime,day_diff,helpful_yes,total_vote
0,A3SBTW3WS4IQSN,B007WTAJTO,NaN,"[0, 0]",No issues.,4.00,Four Stars,1406073600,2014-07-23,138,0,0
1,A18K1ODH1I2MVB,B007WTAJTO,0mie,"[0, 0]","Purchased this for my device, it worked as adv...",5.00,MOAR SPACE!!!,1382659200,2013-10-25,409,0,0
2,A2FII3I2MBMUIA,B007WTAJTO,1K3,"[0, 0]",it works as expected. I should have sprung for...,4.00,nothing to really say....,1356220800,2012-12-23,715,0,0
3,A3H99DFEG68SR,B007WTAJTO,1m2,"[0, 0]",This think has worked out great.Had a diff. br...,5.00,Great buy at this price!!! *** UPDATE,1384992000,2013-11-21,382,0,0
4,A375ZM4U047O79,B007WTAJTO,2&amp;1/2Men,"[0, 0]","Bought it with Retail Packaging, arrived legit...",5.00,best deal around,1373673600,2013-07-13,513,0,0


In [14]:
# Veri setimizde kaç ürün olduğuna bakalım;
df["asin"].value_counts()

# Sadece 1 ürün olduğunu görüyoruz.

B007WTAJTO    4915
Name: asin, dtype: int64

In [15]:
# Ortalama puana hızlıca bakalım;
df["overall"].mean()

4.587589013224822

In [17]:
# Veri setinin betimsel istatistiklerine bakalım;

df.describe().T

,count,mean,std,min,25%,50%,75%,max
overall,4915.00,4.59,1.00,1.00,5.00,5.00,5.00,5.00
unixReviewTime,4915.00,1379465001.67,15818574.32,1339200000.00,1365897600.00,1381276800.00,1392163200.00,1406073600.00
day_diff,4915.00,437.37,209.44,1.00,281.00,431.00,601.00,1064.00
helpful_yes,4915.00,1.31,41.62,0.00,0.00,0.00,0.00,1952.00
total_vote,4915.00,1.52,44.12,0.00,0.00,0.00,0.00,2020.00


    Yukarıdaki date_diff değişkeninden görülebildiği kadar, değerlendirmeden itibaren geçen gün sayısı minimum değeri 1 iken, maksimum değeri 1952'ymiş yani. Yani en eski değerlendirme 1952 gün önce yapılırken en son değerlendirme 1 gün önce yapılmış.

In [18]:
# Değerlendirmeden itibaren geçen gün sayısına göre ağırlıklı puan ortalamasını hesaplayalım;

print(df.loc[df["day_diff"] <= df["day_diff"].quantile(0.25), 
       "overall"].mean())

print(df.loc[(df["day_diff"] > df["day_diff"].quantile(0.25)) & (df["day_diff"] <= df["day_diff"].quantile(0.50)), 
       "overall"].mean())

print(df.loc[(df["day_diff"] > df["day_diff"].quantile(0.50)) & (df["day_diff"] <= df["day_diff"].quantile(0.75)), 
       "overall"].mean())

print(df.loc[(df["day_diff"] > df["day_diff"].quantile(0.75)), "overall"].mean())


4.6957928802588995
4.636140637775961
4.571661237785016
4.4462540716612375


    Yukarıda görüldüğü üzere bu ürün son zamanlarda daha olumlu değerlendirmeler almış.

In [19]:
# İlgili zaman aralıklarına ağırlı verebileceğimzi bir fonksiyon tanımlayalım;

def time_based_weighted_average(dataframe, w1=28, w2=26, w3=24, w4=22):
    return dataframe.loc[dataframe["day_diff"] <= dataframe["day_diff"].quantile(0.25), "overall"].mean() * w1 / 100 + \
           dataframe.loc[(dataframe["day_diff"] > dataframe["day_diff"].quantile(0.25)) & (dataframe["day_diff"] <= dataframe["day_diff"].quantile(0.50)), "overall"].mean() * w2 / 100 + \
           dataframe.loc[(dataframe["day_diff"] > dataframe["day_diff"].quantile(0.50)) & (dataframe["day_diff"] <= dataframe["day_diff"].quantile(0.75)), "overall"].mean() * w3 / 100 + \
           dataframe.loc[(dataframe["day_diff"] > dataframe["day_diff"].quantile(0.75)), "overall"].mean() * w4 / 100


In [20]:
# Fonksiyonumuzu çalıştıralım;

time_based_weighted_average(df, w1=28, w2=26, w3=24, w4=22)

4.595593165128118

In [22]:
# direk ortalamaya bakalım;

df["overall"].mean()

4.587589013224822

### Görev 2:

In [23]:
# "helpful_yes" gibi "helpful_no" Değişkenini üretelim;

df["helpful_no"] = df["total_vote"] - df["helpful_yes"]

In [24]:
# df'imizi sadeleştirelim;

df = df[["reviewerName", "overall", "summary", "helpful_yes", "helpful_no", "total_vote", "reviewTime"]]

# Yeni df'e hızlı bir bakış atalım;
df.head()

,reviewerName,overall,summary,helpful_yes,helpful_no,total_vote,reviewTime
0,NaN,4.00,Four Stars,0,0,0,2014-07-23
1,0mie,5.00,MOAR SPACE!!!,0,0,0,2013-10-25
2,1K3,4.00,nothing to really say....,0,0,0,2012-12-23
3,1m2,5.00,Great buy at this price!!! *** UPDATE,0,0,0,2013-11-21
4,2&amp;1/2Men,5.00,best deal around,0,0,0,2013-07-13


In [25]:
# score_pos_neg_diff, score_average_rating ve wilson_lower_bound skorlarını hesaplayıp veriye ekleyelim.

# Şimdi "wilson_lower_bound" için bir fonksiyon tanımlayalım;

def wilson_lower_bound(up, down, confidence=0.95):
    """
    Wilson Lower Bound Score hesapla

    - Bernoulli parametresi p için hesaplanacak güven aralığının alt sınırı WLB skoru olarak kabul edilir.
    - Hesaplanacak skor ürün sıralaması için kullanılır.
    - Not:
    Eğer skorlar 1-5 arasıdaysa 1-3 negatif, 4-5 pozitif olarak işaretlenir ve bernoulli'ye uygun hale getirilebilir.
    Bu beraberinde bazı problemleri de getirir. Bu sebeple bayesian average rating yapmak gerekir.

    Parameters
    ----------
    up: int
        up count
    down: int
        down count
    confidence: float
        confidence

    Returns
    -------
    wilson score: float

    """
    n = up + down
    if n == 0:
        return 0
    z = st.norm.ppf(1 - (1 - confidence) / 2)
    phat = 1.0 * up / n
    return (phat + z * z / (2 * n) - z * math.sqrt((phat * (1 - phat) + z * z / (4 * n)) / n)) / (1 + z * z / n)


# score_pos_neg_diff için de bir fonksiyon tanımlayalım;
def score_up_down_diff(up, down):
    return up - down

# score_average_rating için bir fonksiyon tanımlayalım;
def score_average_rating(up, down):
    if up + down == 0:
        return 0
    return up / (up + down)


In [26]:
# score_pos_neg_diff'e bakalım

df["score_pos_neg_diff"] = df.apply(lambda x: score_up_down_diff(x["helpful_yes"], x["helpful_no"]), axis=1)
df.sort_values("score_pos_neg_diff", ascending=False).head(10)

,reviewerName,overall,summary,helpful_yes,helpful_no,total_vote,reviewTime,score_pos_neg_diff
2031,"Hyoun Kim ""Faluzure""",5.00,UPDATED - Great w/ Galaxy S4 & Galaxy Tab 4 10...,1952,68,2020,2013-01-05,1884
4212,SkincareCEO,1.00,1 Star reviews - Micro SDXC card unmounts itse...,1568,126,1694,2013-05-08,1442
3449,NLee the Engineer,5.00,Top of the class among all (budget-priced) mic...,1428,77,1505,2012-09-26,1351
317,"Amazon Customer ""Kelly""",1.00,"Warning, read this!",422,73,495,2012-02-09,349
3981,"R. Sutton, Jr. ""RWSynergy""",5.00,"Resolving confusion between ""Mobile Ultra"" and...",112,27,139,2012-10-22,85
4596,"Tom Henriksen ""Doggy Diner""",1.00,Designed incompatibility/Don't support SanDisk,82,27,109,2012-09-22,55
1835,goconfigure,5.00,I own it,60,8,68,2014-02-28,52
4672,Twister,5.00,Super high capacity!!! Excellent price (on Am...,45,4,49,2014-07-03,41
4306,Stellar Eller,5.00,Awesome Card!,51,14,65,2012-09-06,37
315,"Amazon Customer ""johncrea""",5.00,Samsung Galaxy Tab2 works with this card if re...,38,10,48,2012-08-13,28


In [27]:
# score_average_rating'e bakalım;

df["score_average_rating"] = df.apply(lambda x: score_average_rating(x["helpful_yes"], x["helpful_no"]), axis=1)
df.sort_values("score_average_rating", ascending=False).head(10)

,reviewerName,overall,summary,helpful_yes,helpful_no,total_vote,reviewTime,score_pos_neg_diff,score_average_rating
4277,S. Q.,5.00,Perfect!!,1,0,1,2012-12-19,1,1.00
2881,Lou Thomas,5.00,Nexus One Loves This Card!,1,0,1,2012-01-10,1,1.00
1073,C. Sanchez,5.00,Tons of space for phone,1,0,1,2013-08-13,1,1.00
445,"Apache ""Elizabeth""",4.00,Amazon Great Prices,1,0,1,2013-12-18,1,1.00
3923,Rock Your Roots,5.00,What more to say?,1,0,1,2013-12-30,1,1.00
435,Anthony L cate,5.00,Love the extra storage,1,0,1,2012-07-24,1,1.00
2901,luis,5.00,Awesome and fast card :),1,0,1,2013-05-13,1,1.00
2204,"jbwam ""jbwam""",2.00,Sandisk will replace failures due to bad batch...,1,0,1,2013-06-14,1,1.00
2206,JCBiker,5.00,Great card,1,0,1,2013-10-31,1,1.00
3408,"Neng Vang ""Neng2012""",5.00,working no problem,1,0,1,2013-07-25,1,1.00


    Bu veri seti için "score_average_rating" hiç de anlamlı çıkmadı.

In [29]:
# wilson_lower_bound'a bakalım;
df["wilson_lower_bound"] = df.apply(lambda x: wilson_lower_bound(x["helpful_yes"], x["helpful_no"]), axis=1)
df.sort_values("wilson_lower_bound", ascending=False).head(10)

,reviewerName,overall,summary,helpful_yes,helpful_no,total_vote,reviewTime,score_pos_neg_diff,score_average_rating,wilson_lower_bound
2031,"Hyoun Kim ""Faluzure""",5.00,UPDATED - Great w/ Galaxy S4 & Galaxy Tab 4 10...,1952,68,2020,2013-01-05,1884,0.97,0.96
3449,NLee the Engineer,5.00,Top of the class among all (budget-priced) mic...,1428,77,1505,2012-09-26,1351,0.95,0.94
4212,SkincareCEO,1.00,1 Star reviews - Micro SDXC card unmounts itse...,1568,126,1694,2013-05-08,1442,0.93,0.91
317,"Amazon Customer ""Kelly""",1.00,"Warning, read this!",422,73,495,2012-02-09,349,0.85,0.82
4672,Twister,5.00,Super high capacity!!! Excellent price (on Am...,45,4,49,2014-07-03,41,0.92,0.81
1835,goconfigure,5.00,I own it,60,8,68,2014-02-28,52,0.88,0.78
3981,"R. Sutton, Jr. ""RWSynergy""",5.00,"Resolving confusion between ""Mobile Ultra"" and...",112,27,139,2012-10-22,85,0.81,0.73
3807,R. Heisler,3.00,"Good buy for the money but wait, I had an issue!",22,3,25,2013-02-27,19,0.88,0.70
4306,Stellar Eller,5.00,Awesome Card!,51,14,65,2012-09-06,37,0.78,0.67
4596,"Tom Henriksen ""Doggy Diner""",1.00,Designed incompatibility/Don't support SanDisk,82,27,109,2012-09-22,55,0.75,0.66
